In [39]:
import os
import hashlib
import marshal
from pyeoskit import eosapi, wallet, db
from pyeoskit import config
from pyeoskit._hello import _eosapi

db.reset()
config.main_token = 'UUOS'

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')

wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')

def publish_contract(account_name, code, abi):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash != r['code_hash']:
        eosapi.set_contract(account_name, code, abi, 1)
    return True

#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])

def run_test_code(code, account_name='helloworld11'):
    publish_contract(account_name, code, abi)
    try:
        r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'active'})
        print(r['processed']['action_traces'][0]['console'])
    except Exception as e:
        print(e)


def set_code(account_name, code):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash == r['code_hash']:
        return

    setcode = {"account":account_name,
               "vmtype": 1,
               "vmversion":0,
               "code":code.hex()
               }
    eosapi.push_action('eosio', 'setcode', setcode, {account_name:'active'})
    
    return True

def set_abi(account, abi):
    db.set_abi(account, abi)
    abi = _eosapi.pack_abi(abi)
    setabi ={'account':account, 'abi':abi.hex()}
    eosapi.push_action('eosio', 'setabi', setabi, {account:'active'})

print('done!')

done!


In [45]:
code = '''
import struct
from db import MultiIndex
class MyData(object):
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

    def pack(self):
        return struct.pack('QQQ', self.a, self.b, self.c)

    @classmethod
    def unpack(cls, data):
        a, b, c = struct.unpack('QQQ', data)
        return MyData(a, b, c)

    def get_primary_value(self):
        return self.a

    def get_secondary_value(self, index):
        if index == 0:
            return self.b
        elif index == 1:
            return self.c
        raise KeyError

    @classmethod
    def get_secondary_indexes(self):
        return (64, 64)

def apply(receiver, code, action):
    n = N('helloworld11')
    code = scope = table = n
    mi = MultiIndex(code, scope, table, MyData)

    md = MyData(1, 11, 111)
    md.payer = N('helloworld11')
    mi.store(md)

    md = MyData(3, 33, 333)
    md.payer = N('helloworld11')
    mi.store(md)

    print('+++++++++++++++++++++++++++++')
    print(1 in mi)
    print(2 in mi)
    print(3 in mi)
        
    md = mi[1]
    print(md.a, md.b, md.c)

    print('+++++++++++++++++++++++++++++')
    for item in mi:
        print(item.a, item.b, item.c)

    secondary = mi.get_secondary_index(0)
    print('+++++++++++get_secondary_index(0)++++++++++++++++++')
    md = secondary[11]    
    print(md.a, md.b, md.c)
    

    print('+++++++++++get_secondary_index(0)++++++++++++++++++')
    for item in secondary:
        print(item.a, item.b, item.c)

    secondary = mi.get_secondary_index(1)
    print('+++++++++++++get_secondary_index(1)++++++++++++++++')
    md = secondary[333]    
    print(md.a, md.b, md.c)

    print('++++++++++++get_secondary_index(1)+++++++++++++++++')
    for item in secondary:
        print(item.a, item.b, item.c)

'''
abi = ''
name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

+++++++++++++++++++++++++++++
True
False
True
1 11 111
+++++++++++++++++++++++++++++
3 33 333
1 11 111
+++++++++++get_secondary_index(0)++++++++++++++++++
1 11 111
+++++++++++get_secondary_index(0)++++++++++++++++++
3 33 333
1 11 111
+++++++++++++get_secondary_index(1)++++++++++++++++
3 33 333
++++++++++++get_secondary_index(1)+++++++++++++++++
3 33 333
1 11 111

